In [1]:
import requests as req

In [2]:
# Download all index files from EDGAR
def download_files():
    for year in range(1993, 2019):
        for quarter in range(1, 5):
            url = 'https://www.sec.gov/Archives/edgar/full-index/' + str(year) + '/QTR' + str(quarter) + '/company.idx'
            r = req.get(url, allow_redirects=True)
            with open('Index-' + str(year) + '-' + str(quarter) + '.txt', "wb") as f:
                f.write(r.content)

In [3]:
import os

# Reads all index files and returns a dictionary of URLs of all 10-K/10-Qs of the given company
def get_reports(company, startyear, startquarter):
    files = filter(lambda x: x[-4:] == '.txt', os.listdir('.'))
    for file in files:
        year = int(file[6:10])
        if year < startyear:
            continue
        quarter = int(file[11])
        if year == startyear and quarter < startquarter:
            continue
        txt = open(file, 'r')
        for line in txt:
            if line.startswith(company) and ('10-K' in line or '10-Q' in line):
                url_index = line.index('edgar/')
                url = line[url_index:].rstrip('\n').strip()
                yield (year, quarter, 'https://www.sec.gov/Archives/' + url)

In [4]:
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Parses HTML text and returns a generator of all tables as DataFrames
def make_tables(text):
    soup = BeautifulSoup(text)
    tables = soup.find_all("table")
    table_num = 1
    for table in tables:
        row_list = []
        indices = []
        for row in table.find_all('tr'):
            data_row = []
            columns = row.find_all('td')
            for column in columns:
                cell_text = column.get_text().replace('\n','')
                data_row.append(cell_text)
            row_list.append(data_row[1:])
            indices.append(data_row[0])
        if len(row_list) > 0:
            data = pd.DataFrame(row_list, index=indices)
            yield data
            table_num += 1
        if table_num > 12:
            break

In [49]:
# Given a company name, returns list of table generators for all 10-K/10-Qs that are HTML files
def get_html_tables(company, startyear, startquarter):
    urls = get_reports(company, startyear, startquarter)
    for url in urls:
        text = req.get(url[2]).text
        #if '<html' in text or '<HTML' in text:
        data = make_tables(text)
        yield (url[0], url[1], make_tables(text))

items = get_html_tables("APPLE INC", 2018, 1) # Put the company name as listed in the index file
for item in items:
    print('Year:', item[0], 'Quarter', item[1])
    for t in item[2]:
        display(t)

Year: 2018 Quarter 1


""
""
""
""


""
""
""
""


""
""
""
""


""
""
""
""


...

,0,1
,None,None
,,
"Common Stock, $0.00001 par value per share1.000% Notes due 20221.375% Notes due 20240.875% Notes due 20251.625% Notes due 20262.000% Notes due 20271.375% Notes due 20293.050% Notes due 20293.600% Notes due 2042",,The Nasdaq Stock Market LLCNew York Stock Exch...
(Title of each class),,(Name of each exchange on which registered)


""
""
""
""


,0,1,2,3,4,5
,None,None,None,None,None,None
,,,,,,
Large accelerated filer,,☒,,Accelerated filer,,☐
Non-accelerated filer,,☐,,Smaller reporting company,,☐
,,,,Emerging growth company,,☐


""
""
""
""


,0,1
,None,None
,,
,Page,None
Part I,None,None
Item 1.,Business,1
Item 1A.,Risk Factors,8
Item 1B.,Unresolved Staff Comments,17
Item 2.,Properties,18
Item 3.,Legal Proceedings,18
Item 4.,Mine Safety Disclosures,18
